In [1]:
import pandas as pd
import os, math, re
import geopy
from geopy.geocoders import Nominatim
import geocoder

/Users/nikkosiegfried/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/nikkosiegfried/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
df1 = pd.read_csv("data/Film_Locations_in_San_Francisco_20240828.csv")


In [3]:
df1.head(2)

,Title,Release Year,Locations,Fun Facts,Production Company,Distributor,Director,Writer,Actor 1,Actor 2,Actor 3,SF Find Neighborhoods,Analysis Neighborhoods,Current Supervisor Districts
0,Experiment in Terror,1962,The Sea Captain's Chest (Fisherman's Wharf),NaN,Columbia Pictures Corporation,Columbia Pictures,Blake Edwards,The Gordons,Glenn Ford,Lee Remick,Stefanie Powers,99.0,23.0,3.0
1,Experiment in Terror,1962,100 St. Germain Avenue,NaN,Columbia Pictures Corporation,Columbia Pictures,Blake Edwards,The Gordons,Glenn Ford,Lee Remick,Stefanie Powers,47.0,38.0,8.0


In [4]:
#ASSIGN UNIQUE ID, CREATE EMPTY COLUMNS: 'lat', 'lon', 'address', 'query_location'
df1['unique_id'] = range(1, len(df1) + 1)
df1 = df1[['unique_id'] + [col for col in df1.columns if col != 'unique_id']]


df1['lat1'] = None
df1['lon1'] = None
df1['address1'] = None
df1['lat2'] = None
df1['lon2'] = None
df1['address2'] = None
df1['query_location'] = None
df1 = df1[['unique_id'] + [col for col in df1.columns if col != 'unique_id']]


In [5]:
df1['city'] = "San Francisco"
df1['state'] = 'California'

In [6]:
df1.head(2)

,unique_id,Title,Release Year,Locations,Fun Facts,Production Company,Distributor,Director,Writer,Actor 1,...,Current Supervisor Districts,lat1,lon1,address1,lat2,lon2,address2,query_location,city,state
0,1,Experiment in Terror,1962,The Sea Captain's Chest (Fisherman's Wharf),NaN,Columbia Pictures Corporation,Columbia Pictures,Blake Edwards,The Gordons,Glenn Ford,...,3.0,None,None,None,None,None,None,None,San Francisco,California
1,2,Experiment in Terror,1962,100 St. Germain Avenue,NaN,Columbia Pictures Corporation,Columbia Pictures,Blake Edwards,The Gordons,Glenn Ford,...,8.0,None,None,None,None,None,None,None,San Francisco,California


In [7]:
#RETREIVE DF WITHOUT NAN IN LOCATIONS
df_filtered = df1.dropna(subset=['Locations'])


In [8]:
df_filtered.head(2)

,unique_id,Title,Release Year,Locations,Fun Facts,Production Company,Distributor,Director,Writer,Actor 1,...,Current Supervisor Districts,lat1,lon1,address1,lat2,lon2,address2,query_location,city,state
0,1,Experiment in Terror,1962,The Sea Captain's Chest (Fisherman's Wharf),NaN,Columbia Pictures Corporation,Columbia Pictures,Blake Edwards,The Gordons,Glenn Ford,...,3.0,None,None,None,None,None,None,None,San Francisco,California
1,2,Experiment in Terror,1962,100 St. Germain Avenue,NaN,Columbia Pictures Corporation,Columbia Pictures,Blake Edwards,The Gordons,Glenn Ford,...,8.0,None,None,None,None,None,None,None,San Francisco,California


In [9]:
#CONVERT LOCATION DATA INTO USABLE SEARCH QUERIES
#First convert 'low-hanging fruit': extract numbers followed by a word and street type
#eg. '123 main st'

def find_full_street_addresses(text):
    pattern = re.compile(r'(?<![a-zA-Z])\d+\s\w+\s(?:Street|St|Avenue|Ave|Boulevard|Blvd|Road|Rd|Drive|Dr|Lane|Ln|Court|Ct|Place|Pl|Way|Parkway|Pkwy|Square|Sq|Terrace|Ter|Circle|Cir|Alley|Arcade)')
    matches = pattern.findall(text)
    return matches
        
for index, row in df_filtered.iterrows():
    location = row['Locations']
    query = find_full_street_addresses(location)
    if query != []:
        df_filtered.at[index, 'query_location'] = query[0]



In [10]:
#extract numbers followed by two words and street type
#eg. '123 south main st'
def find_two_word_street_addresses(text):
    # Regular expression pattern to match a number followed by two words and a street type
    pattern = re.compile(r'(?<![a-zA-Z])\d+\s\w+\.?\s\w+\.?\s(?:Street|St|Avenue|Ave|Boulevard|Blvd|Road|Rd|Drive|Dr|Lane|Ln|Court|Ct|Place|Pl|Way|Parkway|Pkwy|Square|Sq|Terrace|Ter|Circle|Cir|Alley|Arcade)')
    matches = pattern.findall(text)
    return matches

for index, row in df_filtered[df_filtered['query_location'].isnull()].iterrows():
    location = row['Locations']
    query = find_two_word_street_addresses(location)
    if query != []:
        df_filtered.at[index, 'query_location'] = query[0]

In [11]:
def find_excluding_specific_words(locations):
    # Regular expression pattern to match strings that do not contain 'between', 'btwn', 'and', or 'at'
    pattern = re.compile(r'^(?!.*\b(between|btwn|and|at)\b).+$', re.IGNORECASE)
    matches = [location for location in locations if pattern.search(location)]
    return matches

In [12]:
#extract locations that have streets between two streets and convert them to intersection locations
#eg. 'Clay between Stockton and Grant' -> 'Clay and Stockton'

pattern = re.compile(r'^(.*?)\s+\b(?:between|btwn)\b\s+\b(\w+)\b', re.IGNORECASE)

for index, row in df_filtered[df_filtered['query_location'].isnull()].iterrows():
    location = row['Locations']
    if ' between '  in location or ' btwn ' in location:
        query = pattern.search(location)
        df_filtered.at[index, 'query_location'] = f"{query.group(1)} and {query.group(2)}"

In [13]:
def has_no_street_type(location):
    street_types = r'\b(?:Streets|Street|St|Avenue|Ave|Boulevard|Blvd|Road|Rd|Drive|Dr|Lane|Ln|Court|Ct|Place|Pl|Way|Parkway|Pkwy|Square|Sq|Terrace|Ter|Circle|Cir|Alley|Arcade)\b'
    pattern = re.compile(street_types, re.IGNORECASE)
    
    return not bool(pattern.search(location))

In [14]:
#extract location within parentheses that is not a street name
#eg. Danilo's Bakery (Pier 39)

street_types = r'\b(?:Streets|Street|St|Avenue|Ave|Boulevard|Blvd|Road|Rd|Drive|Dr|Lane|Ln|Court|Ct|Place|Pl|Way|Parkway|Pkwy|Sq|Terrace|Ter|Circle|Cir|Alley|Arcade)\b'

for index, row in df_filtered[df_filtered['query_location'].isnull()].iterrows():
    location = row['Locations']
    if '('  in location and ')'  in location:
        query1 = re.search(r'\((.*?)\)', location).group(1)
        if has_no_street_type(query1):
            df_filtered.at[index, 'query_location'] = query1


In [15]:
#extract location without street types
#eg. Burns Park
for index, row in df_filtered[df_filtered['query_location'].isnull()].iterrows():
    location = row['Locations']
    if has_no_street_type(location):
        df_filtered.at[index, 'query_location'] = location

In [16]:
for index, row in df_filtered[df_filtered['query_location'].isnull()].iterrows():
    location = row['Locations']
    if ' at ' in location:
        query = location.replace('at','&')
        df_filtered.at[index, 'query_location'] = query
    if ' @ ' in location:
        query = location.replace('@','&')
        df_filtered.at[index, 'query_location'] = query       

In [17]:
def has_street_type_before_and(location):
    # Regular expression to identify a street type followed by the word "and"
    street_types = r'\b(?:Street|St|Avenue|Ave|Boulevard|Blvd|Road|Rd|Drive|Dr|Lane|Ln|Court|Ct|Place|Pl|Way|Parkway|Pkwy|Square|Sq|Terrace|Ter|Circle|Cir|Alley|Arcade)\b'
    pattern = re.compile(rf'{street_types}\s+(?:and|&)', re.IGNORECASE)

    # Return True if there is a street type before "and", False otherwise
    return bool(pattern.search(location))

In [18]:
for index, row in df_filtered[df_filtered['query_location'].isnull()].iterrows():
    location = row['Locations']
    if has_street_type_before_and(location):
        df_filtered.at[index, 'query_location'] = location

In [19]:
def add_street_before_ampersand(location):
    # Regular expression to match "&" and add "Street" before it
    pattern = re.compile(r'\s*&', re.IGNORECASE)
    
    # Substitute the pattern with " Street &"
    modified_location = pattern.sub(' street &', location)
    
    return modified_location

In [20]:
#if 'Streets' and '&' in location text, add street before '&' and replace 'Streets' with 'street' 
#eg. '123 Lombard & Delores Streets' -> '123 Lombard street & Delores street'
for index, row in df_filtered[df_filtered['query_location'].isnull()].iterrows():
    location = row['Locations']
    if 'Streets' in location and '&' in location:
        query = add_street_before_ampersand(location)
        df_filtered.at[index, 'query_location'] = query.replace('Streets', 'street')

In [21]:
def extract_text_before_park_until_comma(location):
    # Regular expression to match text before "Park" up to a comma, if present
    pattern = re.compile(r'^(.*?,)?\s*(.*?\s+Park)', re.IGNORECASE)
    
    # Search for the pattern and return the desired text
    match = pattern.search(location)
    if match:
        # If a comma is found before "Park", return the text before and including "Park" after the last comma
        if match.group(1):
            return match.group(2)
        # If no comma is found, return the entire matched text including "Park"
        return match.group(2)
    else:
        return location
def extract_text_after_comma(location):
    # Regular expression to match text after the first comma
    pattern = re.compile(r',\s*(.*)', re.IGNORECASE)
    
    # Search for the pattern and return the text after the comma
    match = pattern.search(location)
    if match:
        return match.group(1)  # Return the text after the comma
    else:
        return location

In [22]:
#extract park names only
#eg. "Hagiwara Tea Garden Drive, Golden Gate Park" -> "Golden Gate Park"
for index, row in df_filtered[df_filtered['query_location'].isnull()].iterrows():
    location = row['Locations']
    if ' Park' in location:
        query = location.split(',')
        if len(query) ==1:
            df_filtered.at[index, 'query_location'] = query[0]
        else:
            for text in query:
                if 'Park' in text:
                    query = extract_text_before_park_until_comma(text)
                    if query != '':
                        df_filtered.at[index, 'query_location'] = query.strip()
                

In [23]:
def is_street_name_followed_by_type(location):
    # Regular expression to match a street name followed by a street type
    street_types = r'\b(?:Street|St|St.|Avenue|Ave|Boulevard|Blvd|Road|Rd|Drive|Dr|Lane|Ln|Court|Ct|Place|Pl|Way|Parkway|Pkwy|Square|Sq|Terrace|Ter|Circle|Cir|Alley|Arcade)\b'
    pattern = re.compile(rf'^[A-Za-z0-9\s]+(?:{street_types})$', re.IGNORECASE)
    
    # Return True if the location matches the pattern, False otherwise
    return bool(pattern.match(location))

In [24]:
# Search for locations with street names followed by a street type
# eg. 'grape st'
for index, row in df_filtered[df_filtered['query_location'].isnull()].iterrows():
    location = row['Locations']
    if is_street_name_followed_by_type(location):
        df_filtered.at[index, 'query_location'] = location

In [25]:
def add_street_before_ampersand_if_needed(location):
    # Regular expression to match "&" not preceded by "St." or "Street"
    pattern = re.compile(r'(\bSt\.\b|\bStreet\b)?\s*&', re.IGNORECASE)
    
    # Check if "St." or "Street" is present before "&"
    if pattern.search(location):
        # Replace "&" with "Street &" if "St." or "Street" is not already before "&"
        location = re.sub(r'(\bSt\.\b|\bStreet\b)?\s*&', ' Street &', location, flags=re.IGNORECASE)
    
    return location

def extract_word_before_ampersand(location):
    # Regular expression to match the word before "&"
    pattern = re.compile(r'(\S+)\s*&', re.IGNORECASE)
    
    # Search for the pattern and return the word before "&"
    match = pattern.search(location)
    if match:
        return match.group(1)
    else:
        return None

In [26]:

for index, row in df_filtered[df_filtered['query_location'].isnull()].iterrows():
    location = row['Locations']
    if '&' in location:
        before_and = extract_word_before_ampersand(location)
        if before_and != 'St.':
            query = add_street_before_ampersand_if_needed(location)
            df_filtered.at[index, 'query_location'] =query
        else:
            df_filtered.at[index, 'query_location'] =location


In [27]:

for index, row in df_filtered[df_filtered['query_location'].isnull()].iterrows():
    location = row['Locations']
    if "Alamo Square" in location:
        df_filtered.at[index, 'query_location'] ="Alamo Square" 
    if "Aladdin Terrace" in location:
        df_filtered.at[index, 'query_location'] ="Aladdin Terrace"
    if "Treasure Island" in location:
        df_filtered.at[index, 'query_location'] ="Treasure Island"        

In [28]:
#change old ATT Stadium to address
for index, row in df_filtered.iterrows():
    location = row['query_location']
    if location =='AT&T Stadium':
        df_filtered.at[index, 'query_location'] ="24 Willie Mays Plaza"

In [29]:
for index, row in df_filtered[df_filtered['query_location'].isnull()].iterrows():
    location = row['Locations']
    df_filtered.at[index, 'query_location'] =location
    

In [30]:
df1[df1['Locations'].isnull()]

,unique_id,Title,Release Year,Locations,Fun Facts,Production Company,Distributor,Director,Writer,Actor 1,...,Current Supervisor Districts,lat1,lon1,address1,lat2,lon2,address2,query_location,city,state
4,5,The Californians,2005,NaN,NaN,Parker Film Company,Fabrication Films,Jonathan Parker,Jonathan Parker & Catherine DiNapoli,Noah Wyle,...,10.0,None,None,None,None,None,None,None,San Francisco,California
5,6,Babies,2010,NaN,NaN,Canal+,Focus Features,Thomas Balmes,Thomas Balmes,Bayar,...,10.0,None,None,None,None,None,None,None,San Francisco,California
23,24,Down Periscope,1996,NaN,NaN,Twentieth Century Fox Film Corp.,Twentieth Century Fox Film Corp.,David S. Ward,Hugh Wilson,Kelsey Grammar,...,10.0,None,None,None,None,None,None,None,San Francisco,California
40,41,Shattered,1991,NaN,NaN,Capella International,Metro-Goldwyn-Mayer,Wolfgang Petersen,Wolfgang Petersen,Tom Berenger,...,10.0,None,None,None,None,None,None,None,San Francisco,California
60,61,True Believer,1989,NaN,NaN,Columbia Pictures Corporation,Columbia Pictures,Joseph Ruben,Wesley Strick,James Woods,...,10.0,None,None,None,None,None,None,None,San Francisco,California
77,78,The Doctor,1991,NaN,NaN,Silver Screen Partners IV,Buena Vista Pictures,Randa Haines,Robert Caswell,William Hurt,...,10.0,None,None,None,None,None,None,None,San Francisco,California
130,131,Fathers' Day,1997,NaN,NaN,Northern Lights Entertainment,Columbia Broadcasting System,Ivan Reitman,Lowell Ganz,Robin Williams,...,10.0,None,None,None,None,None,None,None,San Francisco,California
273,274,The Organization,1971,NaN,NaN,The Mirisch Corporation,United Artists,Don Medford,James R. Webb,Sidney Poitier,...,10.0,None,None,None,None,None,None,None,San Francisco,California
280,281,Raising Cain,1992,NaN,NaN,Pacific Western,Universal Pictures,Brian DePalma,Brian DePalma,John Lithgow,...,10.0,None,None,None,None,None,None,None,San Francisco,California
290,291,Just Like Heaven,2005,NaN,NaN,DreamWorks SKG,DreamWorks Distribution,Mark Waters,Peter Tolan,Reese Witherspoon,...,10.0,None,None,None,None,None,None,None,San Francisco,California


In [31]:
df1_final = pd.concat([df1[df1['Locations'].isnull()], df_filtered], ignore_index=True)


In [32]:
df1_final

,unique_id,Title,Release Year,Locations,Fun Facts,Production Company,Distributor,Director,Writer,Actor 1,...,Current Supervisor Districts,lat1,lon1,address1,lat2,lon2,address2,query_location,city,state
0,5,The Californians,2005,NaN,NaN,Parker Film Company,Fabrication Films,Jonathan Parker,Jonathan Parker & Catherine DiNapoli,Noah Wyle,...,10.0,None,None,None,None,None,None,None,San Francisco,California
1,6,Babies,2010,NaN,NaN,Canal+,Focus Features,Thomas Balmes,Thomas Balmes,Bayar,...,10.0,None,None,None,None,None,None,None,San Francisco,California
2,24,Down Periscope,1996,NaN,NaN,Twentieth Century Fox Film Corp.,Twentieth Century Fox Film Corp.,David S. Ward,Hugh Wilson,Kelsey Grammar,...,10.0,None,None,None,None,None,None,None,San Francisco,California
3,41,Shattered,1991,NaN,NaN,Capella International,Metro-Goldwyn-Mayer,Wolfgang Petersen,Wolfgang Petersen,Tom Berenger,...,10.0,None,None,None,None,None,None,None,San Francisco,California
4,61,True Believer,1989,NaN,NaN,Columbia Pictures Corporation,Columbia Pictures,Joseph Ruben,Wesley Strick,James Woods,...,10.0,None,None,None,None,None,None,None,San Francisco,California
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2079,2080,Vertigo,1958,900 Lombard Street,Lombard Street is not actually the most crooke...,Alfred J. Hitchcock Productions,Paramount Pictures,Alfred Hitchcock,Alec Coppel,James Stewart,...,6.0,None,None,None,None,None,None,900 Lombard Street,San Francisco,California
2080,2081,Love & Taxes,2014,The Marsh Theatre (1062 Valencia Street),NaN,Bad Company Films,NaN,Jacob Kornbluth,Jacob Kornbluth,Jacob Kornbluth,...,5.0,None,None,None,None,None,None,1062 Valencia Street,San Francisco,California
2081,2082,Chance- Season 1 ep109,2016,Terry A. Francois Blvd.,NaN,TVM Productions,HULU,Sara Gran and Pete Begler,Dan Attias,Hugh Laurie,...,10.0,None,None,None,None,None,None,Terry A. Francois Blvd.,San Francisco,California
2082,2083,Women is Losers,2020,1132 Florida St,NaN,Look at the Moon Pictures,NaN,Lissette Feliciano,Lissette Feliciano,Lorenza Izzo,...,2.0,None,None,None,None,None,None,1132 Florida St,San Francisco,California


In [33]:
df1_final.to_csv('data/Film_Locations_Cleaned_Step1.csv', index=False)
